In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2016,2023))

In [3]:
SEASONS

[2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [76]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR,"standings")
SCORES_DIR = os.path.join(DATA_DIR,"scores")

In [71]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html
    

In [59]:
url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
html = await get_html(url,"#content .filter")

2015-16 NBA Schedule | Basketball-Reference.com


In [74]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)
    

In [83]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Refere

In [108]:
standings_files = os.listdir(STANDINGS_DIR)

In [110]:
standings_files

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2019_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2016_games-april.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2017_games-february.html',
 'NBA_2016_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2020_games-october-2019.html',
 'NBA_2022_games-december.html',
 'NBA_2017_games-april.html',
 'NBA_2022_games-november.html',
 'NBA_2016_games-january.html',
 'NBA_2018_games-october.html',
 'NBA_2017_games-march.html',
 'NBA_2021_games-february.html',
 'NBA_2020_games-fe

In [111]:
async def scrape_game(standings_file):
    with open(standings_file,'r') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
    
        html = await get_html(url,"#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [112]:
for f in standing_files:
    filepath = os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)

Celtics vs Rockets, October 24, 2021 | Basketball-Reference.com
Magic vs Knicks, October 24, 2021 | Basketball-Reference.com
76ers vs Thunder, October 24, 2021 | Basketball-Reference.com
Warriors vs Kings, October 24, 2021 | Basketball-Reference.com
Grizzlies vs Lakers, October 24, 2021 | Basketball-Reference.com
Celtics vs Hornets, October 25, 2021 | Basketball-Reference.com
Bucks vs Pacers, October 25, 2021 | Basketball-Reference.com
Bulls vs Raptors, October 25, 2021 | Basketball-Reference.com
Magic vs Heat, October 25, 2021 | Basketball-Reference.com
Wizards vs Nets, October 25, 2021 | Basketball-Reference.com
Pistons vs Hawks, October 25, 2021 | Basketball-Reference.com
Pelicans vs Timberwolves, October 25, 2021 | Basketball-Reference.com
Cavaliers vs Nuggets, October 25, 2021 | Basketball-Reference.com
Trail Blazers vs Clippers, October 25, 2021 | Basketball-Reference.com
76ers vs Knicks, October 26, 2021 | Basketball-Reference.com
Warriors vs Thunder, October 26, 2021 | Basketba

Error: Page.goto: NS_ERROR_UNKNOWN_HOST
Call log:
  - navigating to "https://www.basketball-reference.com/boxscores/202102110HOU.html", waiting until "load"
